## Script to update FHIR Questionnaires questions

- python version 3.6+

- upload example as python dict from file
- update questions
- write to FHIR server as json

In [265]:
from json import load, dumps
from flatten_json import flatten
import pandas as pd
from csv import writer
import openpyxl
from IPython import display as D
from fhirclient.models import questionnaire as Q
from fhirclient.models import extension as X
from fhirclient.models import valueset as VS
from fhirclient.models import fhirdate as FD
from requests import get, post, put
from datetime import datetime, date

In [266]:

now = f'{datetime.utcnow().isoformat()}Z'
today = str(date.today())
out_path = ''
out_file = f'{in_file}C'


### Open Questionnaires as DICT files

In [267]:
def open_file(in_path, name): # get files
    with open(f'{in_path}{name}.json') as f:
        r = load(f)
        return(r)

### get file
- update id  
- update url
- update identifier
- update title
- scrub text
- update description





In [268]:
in_path = '/Users/ehaas/Documents/FHIR/Argo-Questionnaire/source/examples/archive/'

#files = ['ASQ3','AUDIT','DAST','Housing','PHQ9','Sampler']

in_file = 'AUDIT'

q = Q.Questionnaire(open_file(in_path, in_file))

#print(q.id)
q.id = f'{q.id}-c'
q.url = f'{q.url}-c'
q.identifier[0].value = f'{q.identifier[0].value}-c'
q.title = 'Alcohol Use Disorder Identification Test - Consumption [AUDIT-C]'
q.date = FD.FHIRDate(today)
q.code[0].code = '72109-2'
q.code[0].display = 'Alcohol Use Disorder Identification Test - Consumption [AUDIT-C]'
#print(q.id, q.url, q.identifier[0].as_json(), q.title, q.date)
#print(q.meta.as_json())
q.text = None
q.description='The Alcohol Use Disorders Identification Test-Consumption (AUDIT-C) is a brief validated screen for risky drinking and alcohol abuse and dependence (alcohol misuse).'



#### replace contains

In [269]:
q.contained = []
ids = ['LL2179-1','LL2180-9','LL2181-7']

for c in ids:
    cvs = VS.ValueSet(open_file(in_path='',name=c))
    q.contained.append(cvs)
    
print(dumps(q.as_json(),indent=3))
  

{
   "id": "questionnaire-example-audit-c",
   "meta": {
      "profile": [
         "http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/q"
      ]
   },
   "contained": [
      {
         "id": "LL2179-1",
         "meta": {
            "profile": [
               "http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/qcvs"
            ]
         },
         "compose": {
            "include": [
               {
                  "valueSet": [
                     "http://loinc.org/vs/LL2179-1"
                  ]
               }
            ]
         },
         "copyright": "This content LOINC is copyright 1995 Regenstrief Institute, Inc. and the LOINC Committee, and available at no cost under the license at http://loinc.org/terms-of-use.",
         "expansion": {
            "contains": [
               {
                  "extension": [
                     {
                        "url": "http://hl7.org/fhir/StructureDefinition/valueset-conceptOrder"

#### pop out  questions

In [270]:
q_item = q.item[0].as_json()

for i in reversed([0,4,5,6,7,8,9,10]):
   q_item['item'].pop(i)
    
q.item[0] = Q.QuestionnaireItem(q_item)
print(dumps(q.item[0].as_json(),indent=3))


{
   "extension": [
      {
         "url": "http://hl7.org/fhir/StructureDefinition/questionnaire-hidden",
         "valueBoolean": true
      }
   ],
   "code": [
      {
         "code": "72110-0",
         "system": "http://loinc.org"
      }
   ],
   "item": [
      {
         "code": [
            {
               "code": "68518-0\u00a0\u00a0",
               "system": "http://loinc.org"
            }
         ],
         "linkId": "g1.q1",
         "options": {
            "display": "SAMHSA-6 drinks / Never/Less than monthly/Monthly/Weekly/Daily/Almost daily",
            "reference": "#SAMHSA-6"
         },
         "prefix": "1. ",
         "repeats": false,
         "required": true,
         "text": "How often do you have a drink containing alcohol?",
         "type": "choice"
      },
      {
         "code": [
            {
               "code": "68519-8",
               "system": "http://loinc.org"
            }
         ],
         "linkId": "g1.q2",
         "options"

- move quesstions to top level
- change linkID
- change code
- change options reference
- change text

In [271]:
new_q = [
    ('68518-0',
  'How often do you have a drink containing alcohol',
  '#LL2179-1',
  'Never/Monthly or less/2-4 times a month/2-3 times a week/4 or more times a week'),
    ('68519-8',
  'How many standard drinks containing alcohol do you have on a typical day',
  '#LL2180-9',
  '1 or 2 / 3 or 4 / 5 or 6 / 7 to 9 / 10 or more'),
    ('68520-6',
  'How often do you have 6 or more drinks on 1 occasion',
  '#LL2181-7',
  'Never / Less than monthly / Monthly / Weekly / Daily or almost daily')
]

new_d = "The Alcohol Use Disorders Identification Test C (AUDIT-C) is scored on a scale of 0-12 \
where the higher the score, the more likely the patient's drinking is hazardous. A score of 4 or\
more for men and 3 or more for women is considered positive for hazardous drinking or active \
alcohol use disorders. If the points are all from Question 1 alone where 2 and 3 are 0, it is \
likely the patient is drinking below recommended limits. The care provider may review the patients\
alcohol intake over that past few months to confirm accuracy."

qnum, dnum = 1,1
for i,j in enumerate(q.item[0].item):
    if j.type =="choice":
        j.prefix = f'{i}.'
        j.linkId = f'q{qnum}'
        j.code[0].code = new_q[i][0]
        j.code[0].display = new_q[i][1]
        j.options.reference = new_q[i][2]
        j.options.display = new_q[i][3]
        j.text = new_q[i][1]
        qnum += 1
        
    else:
        j.linkId = f'd{dnum}'
        j.text = new_d
        j.required = None
        dnum  += 1
        
    
            
    q.item.append(j)
del q.item[0]
for i in q.item:
    print(dumps(i.as_json(), indent=3))

{
   "code": [
      {
         "code": "68518-0",
         "display": "How often do you have a drink containing alcohol",
         "system": "http://loinc.org"
      }
   ],
   "linkId": "q1",
   "options": {
      "display": "Never/Monthly or less/2-4 times a month/2-3 times a week/4 or more times a week",
      "reference": "#LL2179-1"
   },
   "prefix": "0.",
   "repeats": false,
   "required": true,
   "text": "How often do you have a drink containing alcohol",
   "type": "choice"
}
{
   "code": [
      {
         "code": "68519-8",
         "display": "How many standard drinks containing alcohol do you have on a typical day",
         "system": "http://loinc.org"
      }
   ],
   "linkId": "q2",
   "options": {
      "display": "1 or 2 / 3 or 4 / 5 or 6 / 7 to 9 / 10 or more",
      "reference": "#LL2180-9"
   },
   "prefix": "1.",
   "repeats": false,
   "required": true,
   "text": "How many standard drinks containing alcohol do you have on a typical day",
   "type": "choice"
}


### Validate the Resource
Using the $validate operation, the example is validated by a FHIR Reference Server. The results are displayed below in the human readable text as xhtml.



In [272]:

# *********************** validate Resource ********************************

def validate(r):
    Type = 'Questionnaire'
    
    fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
    'profile': r.meta.profile[0]
    }
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/{Type}/$validate', params = params, headers = headers, data = dumps(r.as_json()))
    # print(r.status_code)
    # view  output
    return r.json()["text"]["div"]



    

In [273]:
#validate

display(D.HTML(f'<h1>Validation output</h1>{validate(q)}'))

WARNING,[Questionnaire.meta.profile[0]],"StructureDefinition reference ""http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/q"" could not be resolved"
WARNING,[Questionnaire.contained[1].meta.profile[0]],"StructureDefinition reference ""http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/qcvs"" could not be resolved"
WARNING,[Questionnaire.contained[2].meta.profile[0]],"StructureDefinition reference ""http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/qcvs"" could not be resolved"
WARNING,[Questionnaire.contained[3].meta.profile[0]],"StructureDefinition reference ""http://fhir.org/guides/argonaut-questionnaire/StructureDefinition/qcvs"" could not be resolved"
WARNING,[Questionnaire.jurisdiction],"None of the codes provided are in the value set http://hl7.org/fhir/ValueSet/jurisdiction (http://hl7.org/fhir/ValueSet/jurisdiction, and a code should come from this value set unless it has no suitable code) (codes = urn:iso:std:iso:3166#US)"
ERROR,[Questionnaire.code],"The Coding references a value set, not a code system (""http://loinc.org"")"
ERROR,[Questionnaire.item[1].code],"The Coding references a value set, not a code system (""http://loinc.org"")"
ERROR,[Questionnaire.item[2].code],"The Coding references a value set, not a code system (""http://loinc.org"")"
ERROR,[Questionnaire.item[3].code],"The Coding references a value set, not a code system (""http://loinc.org"")"


### Write to file

In [274]:
def write_file(name, data): # write file
    out_path = ''
    with open(f'{out_path}{name}.json', 'w') as f:
        f.write(data)

In [275]:
r = q
rjson = dumps(r.as_json())
# write files as json
write_file(r.id, rjson)